In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r"C:\Users\sayala\Documents\GitHub\InSPIRE\Studies\USMap_Doubleday_2024\agrivoltaics_irradiance_single_location-time-index.csv", index_col=[0])

In [3]:
df.index

Index(['2001-01-01 00:30:00', '2001-01-01 01:30:00', '2001-01-01 02:30:00',
       '2001-01-01 03:30:00', '2001-01-01 04:30:00', '2001-01-01 05:30:00',
       '2001-01-01 06:30:00', '2001-01-01 07:30:00', '2001-01-01 08:30:00',
       '2001-01-01 09:30:00',
       ...
       '2001-12-31 14:30:00', '2001-12-31 15:30:00', '2001-12-31 16:30:00',
       '2001-12-31 17:30:00', '2001-12-31 18:30:00', '2001-12-31 19:30:00',
       '2001-12-31 20:30:00', '2001-12-31 21:30:00', '2001-12-31 22:30:00',
       '2001-12-31 23:30:00'],
      dtype='object', name='time', length=8760)

In [4]:
cw = 2
pitch = 5

In [5]:
underpanel_left_start = 0
underpanel_left_end = cw/2  # u
underpanel_right_start = pitch-cw/2  # b
underpanel_right_end = pitch

In [6]:
dx = pitch/10

In [7]:
# Tracking TILT, 3 beds
xp = cw/2 # Projection of panel on P.
u = int(np.ceil(10*underpanel_left_end/pitch)) # underpanel limit integer box
b = 10-u
# Three testbeds:
e2e = b-u # dimensions of edge to edge
bA = int(np.ceil(e2e/3.0))
bC = int(bA)
bB = int(e2e-bA-bC)

underpanel = df.iloc[:,0:u].join(df.iloc[:,b:10]).mean(axis=1)
edgetoedge = df.iloc[:,u:b].mean(axis=1)
bedA = df.iloc[:,u:u+bA].mean(axis=1)
bedB = df.iloc[:,u+bA:u+bA+bB].mean(axis=1)
bedC = df.iloc[:,u+bA+bB:b].mean(axis=1)

In [8]:
df['underpanel'] = underpanel
df['edgetoedge'] = edgetoedge
df['bedA'] = bedA
df['bedB'] = bedB
df['bedC'] = bedC


In [9]:
import pvlib

In [10]:
NREL_API_KEY = None  # <-- please set your NREL API key here
# note you must use "quotes" around your key as it is a string.

if NREL_API_KEY is None:
       NREL_API_KEY = 'DEMO_KEY'  # OK for this demo, but better to get your own key

In [11]:
import pvlib

metdata, metadata = pvlib.iotools.get_psm3(
    latitude=39.7555, longitude=-105.2211,
#    latitude=44.25, longitude=-117.33999633789062,
    api_key=NREL_API_KEY,
    email='silvana.ovaitt@nrel.gov',  # <-- any email works here fine
    names='tmy', map_variables=True)
metadata

{'Source': 'NSRDB',
 'Location ID': '146288',
 'City': '-',
 'State': '-',
 'Country': '-',
 'Time Zone': -7,
 'Local Time Zone': -7,
 'Dew Point Units': 'c',
 'DHI Units': 'w/m2',
 'DNI Units': 'w/m2',
 'GHI Units': 'w/m2',
 'Temperature Units': 'c',
 'Pressure Units': 'mbar',
 'Wind Direction Units': 'Degrees',
 'Wind Speed Units': 'm/s',
 'Surface Albedo Units': 'N/A',
 'Version': '3.2.0',
 'latitude': 39.77,
 'longitude': -105.22,
 'altitude': 1879}

In [12]:
df['ghi'] = list(metdata.ghi)

In [13]:
df['SF_underpanel'] = (1-df['underpanel']/df['ghi'])*100
df['SF_edgetoedge'] = (1-df['edgetoedge']/df['ghi'])*100
df['SF_bedA'] = (1-df['bedA']/df['ghi'])*100
df['SF_bedB'] = (1-df['bedB']/df['ghi'])*100
df['SF_bedC'] = (1-df['bedC']/df['ghi'])*100

In [14]:
# Only necessary because I don't know the real location so i'm getting neg numbers
df['SF_underpanel'] = df['SF_underpanel'].mask(df['SF_underpanel'] < 0, np.nan)
df['SF_edgetoedge'] = df['SF_edgetoedge'].mask(df['SF_edgetoedge'] < 0, np.nan)
df['SF_bedA'] = df['SF_bedA'].mask(df['SF_bedA'] < 0, np.nan)
df['SF_bedB'] = df['SF_bedB'].mask(df['SF_bedB'] < 0, np.nan)
df['SF_bedC'] = df['SF_bedC'].mask(df['SF_bedC'] < 0, np.nan)


In [17]:
df['PAR_ghi'] = df['ghi']*3600/1000000*0.43*638.89
df['PAR_underpanel'] = df['underpanel']*3600/1000000*0.43*638.89
df['PAR_edgetoedge'] = df['edgetoedge']*3600/1000000*0.43*638.89
df['PAR_bedA'] = df['bedA']*3600/1000000*0.43*638.89
df['PAR_bedB'] = df['bedB']*3600/1000000*0.43*638.89
df['PAR_bedC'] = df['bedC']*3600/1000000*0.43*638.89


In [20]:
df.to_csv('Results_Setup1_example.csv')

## OTHERS

In [ ]:
# FIXED TILT 3 Beds (i.e. # SCENARIO 1 )

xp = CW*np.cos(np.radians(float(meta['Tilt(deg)']))) # Projection of panel on P.
u = int(np.ceil(10*xp/pitch)) # underpanel limit integer box
e2e = 10-u
# Three testbeds:
bA = int(np.ceil(b/3.0))
bC = int(bA)
bB = int(e2e-bA-bC)

underpanel = df.iloc[:,0:u].mean(axis=1)
underpanel.append(ground[mask].iloc[:,0:u].mean(axis=1).mean())
edgetoedge = df.iloc[u:].mean(axis=1)
bedA = df.iloc[:,u:u+bA].mean(axis=1).mean()
bedB = df.iloc[:,u+bA:u+bA+bB].mean(axis=1).mean()
bedC = df.iloc[:,u+bA+bB:].mean(axis=1).mean()

In [ ]:
# Fixed tilt, 6 beds
xp = CW*np.cos(np.radians(float(meta['Tilt(deg)']))) # Projection of panel on P.
u = int(np.ceil(10*xp/pitch)) # underpanel limit integer box
e2e = 10-u
# Three testbeds:
b1 = int(np.ceil(b/6.0))
b2 = int(e2e-b1*5)

underpanel = df.iloc[:,0:u].join(df.iloc[:,b:10]).mean(axis=1)
edgetoedge = df.iloc[u:b].mean(axis=1)
bedA = df.iloc[:,u:u+b1*1].mean(axis=1)
bedB = df.iloc[:,u+b1*1:u+b1*3].mean(axis=1)
bedC = df.iloc[:,u+b1*2:u+b1*4].mean(axis=1)
bedD = df.iloc[:,u+b1*3:u+b1*4+b2].mean(axis=1)
bedE = df.iloc[:,u+b1*4+b2:u+b1*5+b2].mean(axis=1)
bedF = df.iloc[:,u+b1*5+b2:].mean(axis=1)

In [ ]:
# Tracking, 6 beds
xp = cw/2 # Projection of panel on P.
u = int(np.ceil(10*underpanel_left_end/pitch)) # underpanel limit integer box
b = 10-u
# Six testbeds:
e2e = b-u # dimensions of edge to edge
bA = int(np.ceil(e2e/6.0))
bB = int(bA)
bC = int(bA)
bE = int(bA)
bF = int(bA)
bD = int(e2e-bA-bB-bC-bE-bF)

underpanel = df.iloc[:,0:u].join(df.iloc[:,b:10]).mean(axis=1)
edgetoedge = df.iloc[u:b].mean(axis=1)
bedA = df.iloc[:,u:u+bA].mean(axis=1)
bedB = df.iloc[:,u+bA:u+bA+bB].mean(axis=1)
bedC = df.iloc[:,u+bA+bB:u+bA+bB+bC].mean(axis=1)
bedD = df.iloc[:,u+bA+bB+bC:u+bA+bB+bC+bD].mean(axis=1)
bedE = df.iloc[:,u+bA+bB+bC+bD:u+bA+bB+bC+bD+bE].mean(axis=1)
bedF = df.iloc[:,u+bA+bB+bC+bD+bE:b].mean(axis=1)

In [ ]:
# Fixed tilt, VERTICAL BEDS??
pitch = 8.6
xp = 0.6 #m Spacing from panels to not harvest and damage 
u = int(np.ceil(10*underpanel_left_end/pitch)) # underpanel limit integer box
b = 10-u
# Six testbeds:
e2e = b-u # dimensions of edge to edge
bA = int(np.ceil(e2e/6.0))
bB = int(bA)
bC = int(bA)
bE = int(bA)
bF = int(bA)
bD = int(e2e-bA-bB-bC-bE-bF)

underpanel = df.iloc[:,0:u].join(df.iloc[:,b:10]).mean(axis=1)
edgetoedge = df.iloc[u:b].mean(axis=1)
bedA = df.iloc[:,u:u+bA].mean(axis=1)
bedB = df.iloc[:,u+bA:u+bA+bB].mean(axis=1)
bedC = df.iloc[:,u+bA+bB:u+bA+bB+bC].mean(axis=1)
bedD = df.iloc[:,u+bA+bB+bC:u+bA+bB+bC+bD].mean(axis=1)
bedE = df.iloc[:,u+bA+bB+bC+bD:u+bA+bB+bC+bD+bE].mean(axis=1)
bedF = df.iloc[:,u+bA+bB+bC+bD+bE:b].mean(axis=1)


In [ ]:
# Tracking, 9 beds
xp = cw/2 # Projection of panel on P.
u = int(np.ceil(10*underpanel_left_end/pitch)) # underpanel limit integer box
b = 10-u
# Six testbeds:
e2e = b-u # dimensions of edge to edge
b1 = int(np.ceil(e2e/9.0))
b5 = int(e2e-b1*8)

underpanel = df.iloc[:,0:u].join(df.iloc[:,b:10]).mean(axis=1)
edgetoedge = df.iloc[u:b].mean(axis=1)
bedA = df.iloc[:,u:u+bA].mean(axis=1)
bedB = df.iloc[:,u+bA:u+bA+bB].mean(axis=1)
bedC = df.iloc[:,u+bA+bB:u+bA+bB+bC].mean(axis=1)
bedD = df.iloc[:,u+bA+bB+bC:u+bA+bB+bC+bD].mean(axis=1)
bedE = df.iloc[:,u+bA+bB+bC+bD:u+bA+bB+bC+bD+bE].mean(axis=1)
bedF = df.iloc[:,u+bA+bB+bC+bD+bE:b].mean(axis=1)

In [ ]:
# Tracking, 9 beds
pitch = 11
xp = cw/2 # Projection of panel on P.
u = int(np.ceil(10*underpanel_left_end/pitch)) # underpanel limit integer box
b = 10-u
# Six testbeds:
e2e = b-u # dimensions of edge to edge
b1 = int(np.floor(e2e/8.0))
b5 = int(e2e-b1*7)

underpanel = df.iloc[:,0:u].join(df.iloc[:,b:10]).mean(axis=1)
edgetoedge = df.iloc[u:b].mean(axis=1)
bedA = df.iloc[:,u:u+b1].mean(axis=1)
bedB = df.iloc[:,u+b1*1:u+b1*2].mean(axis=1)
bedC = df.iloc[:,u+b1*2:u+b1*3].mean(axis=1)
bedD = df.iloc[:,u+b1*3:u+b1*4].mean(axis=1)
bedE = df.iloc[:,u+b1*4:u+b1*4+b5].mean(axis=1)
bedF = df.iloc[:,u+b1*4+b5:u+b1*5+b5].mean(axis=1)
bedG = df.iloc[:,u+b1*5+b5:u+b1*6+b5].mean(axis=1)
bedH = df.iloc[:,u+b1*6+b5:u+b1*7+b5].mean(axis=1)


In [ ]:
# Tracking TILT
underpanel_left_start = 0
underpanel_left_end = cw/2  # u
underpanel_right_start = pitch-cw/2  # b
underpanel_right_end = pitch

xp = cw/2 # Projection of panel on P.
u = int(np.ceil(10*underpanel_left_end/pitch)) # underpanel limit integer box
b = 10-u
# Three testbeds:
e2e = b-u # dimensions of edge to edge
bA = int(np.ceil(e2e/3.0))
bC = int(bA)
bB = int(e2e-bA-bC)

underpanel.append((ground[mask].iloc[:,0:u]+ ground[mask].iloc[:,b:10]).mean(axis=1).mean()) 
bedA.append(ground[mask].iloc[:,u:u+bA].mean(axis=1).mean())
bedB.append(ground[mask].iloc[:,u+bA:u+bA+bB].mean(axis=1).mean())
bedC.append(ground[mask].iloc[:,u+bA+bB:b].mean(axis=1).mean())
.mean(axis=1).mean())

In [ ]:
# Calculate geometry
xp = np.cos(np.radians(float(meta['Tilt(deg)']))) # Projection of panel on P.
u = int(np.ceil(100*xp/pitch)) # underpanel limit
b = 100-u   # 
bA = int(np.floor(b/3.0))
bC = int(bA)
bB = int(b-bA-bC)

underpanel = []
bedA = []
bedB = []
bedC = []
for mmonths in range (5, 10):
    datestart = data[data.index.month == mmonths].iloc[0].date
    dateend = data[data.index.month == mmonths].iloc[-1].date
    mask = (data.index >= datestart) & (data.index <= dateend)
    underpanel.append(ground[mask].iloc[:,0:u].mean(axis=1).mean())
    bedA.append(ground[mask].iloc[:,u:u+bA].mean(axis=1).mean())
    bedB.append(ground[mask].iloc[:,u+bA:u+bA+bB].mean(axis=1).mean())
    bedC.append(ground[mask].iloc[:,u+bA+bB:].mean(axis=1).mean())
x = underpanel, bedA, bedB, bedC

results = [meta_dict['latitude'], meta_dict['longitude'], x]